In [1]:
#Install pwntools library
!pip install pwntools
#Clone the assignment problems from GitHub
!git clone https://github.com/ajn313/ML-CyberSec-2025-Lab2-Public
!apt-get -y update
#Clone the radare2 tool repo
!git clone https://github.com/radareorg/radare2
#Install radare2
!cd radare2 && sys/install.sh

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.9/223.9 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 9.8 MB/s eta 0:00:00
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size

In [3]:
#Check radare2 installed correctly
!r2 -v

radare2 6.0.7 34514 @ linux-x86-64
birth: git.6.0.7-51-g1af795bb7d 2025-11-29__05:43:39
commit: 1af795bb7ddc246d3e642e870640a0476e3fc75e
options: gpl -O? cs:5 cl:2 make


In [4]:
#Install r2dec dependencies
!pip install meson ninja
#Update r2pm and install r2dec
!r2pm -U
!r2pm -i r2dec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 17.3 MB/s eta 0:00:00
INFO: git clone --depth=1 --recursive -c core.autocrlf=input https://github.com/radareorg/radare2-pm /root/.local/share/radare2/r2pm/git/radare2-pm
Cloning into '/root/.local/share/radare2/r2pm/git/radare2-pm'...
remote: Enumerating objects: 272, done.
remote: Counting objects: 100% (272/272), done.
remote: Compressing objects: 100% (269/269), done.
remote: Total 272 (delta 26), reused 15 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (272/272), 74.70 KiB | 2.33 MiB/s, done.
Resolving deltas: 100% (26/26), done.
WARN: r2pm database seems older than 14 days: Run 'r2pm -U' to update it
INFO: Using r2-6.0.7 and r2pm-6.0.7
INFO: git clone --depth=1 --recursive -c core.autocrlf=input https://github.com/wargio/r2dec-js /root/.local/share/radare2/r2pm/git/r2dec
Cloning into '/root/.local/share/radare2/r2pm/git/r2dec'...
remote

In [3]:
! pwd

/content


In [15]:
import json
import subprocess
import os
from openai import OpenAI
from google import genai
from google.genai import types

In [ ]:

from google.colab import userdata
api_key = userdata.get('GEMINI_API_KEY')


In [21]:
#Example Instruction: Create a Python file called add.py that adds 4+6, then run the file with python3
# 1. Init client
client = genai.Client(api_key=api_key)

MODEL = "gemini-2.5-flash"  # Corrected model name

SYSTEM_PROMPT = """
You are a command-line automation agent.

You have access to a bash shell via a tool called run_command.

When you want to run a command, you MUST respond with a JSON object ONLY,
no extra text, in this format:

{"action": "run_command", "command": "<command here>"}

When you are done and no more commands are needed, respond with:

{"action": "finish", "message": "<final answer to the user>"}

Guidelines:
- Use simple, safe commands.
- Assume a Unix-like shell.
- Keep commands idempotent when possible.
- Short, focused commands are better than long fragile ones.
"""
def call_gemini(message, chat):
    """Call gemini with given messages and return the response content string."""

    response = chat.send_message(message).text
    return response


def run_shell_command(cmd: str, timeout: int = 20):
    """Run a shell command and return (stdout, stderr, returncode)."""
    try:
        result = subprocess.run(
            cmd,
            shell=True,
            capture_output=True,
            text=True,
            timeout=timeout,
        )
        return result.stdout.strip(), result.stderr.strip(), result.returncode
    except subprocess.TimeoutExpired as e:
        return "", f"Command timed out after {timeout} seconds", -1
    except Exception as e:
        return "", f"Error running command: {e}", -1

def agent_loop(user_goal: str, max_steps: int = 20):
    # Conversation history for GPT
    chat = client.chats.create(model=MODEL,config=types.GenerateContentConfig(system_instruction=SYSTEM_PROMPT))
    message = user_goal

    for step in range(max_steps):
        print(f"\n=== Step {step + 1} ===")
        reply = call_gemini(message, chat)
        print("SENT MESSAGE:", message)
        print("RAW GPT REPLY:", reply)

        # Try to parse JSON
        try:
            action = json.loads(reply)
        except json.JSONDecodeError:
            # If GPT messed up the format, tell it and continue
            message = "You must reply with pure JSON only. Try again."
            continue

        if action.get("action") == "run_command":
            command = action.get("command", "")
            print(f"Executing command: {command!r}")

            stdout, stderr, rc = run_shell_command(command)
            result_summary = (
                f"Command: {command}\n"
                f"Return code: {rc}\n"
                f"STDOUT:\n{stdout or '(empty)'}\n\n"
                f"STDERR:\n{stderr or '(empty)'}"
            )

            # Add tool result back into conversation
            message = "Here is the result of your command:\n" + result_summary

        elif action.get("action") == "finish":
            final_msg = action.get("message", "")
            print("\n=== FINAL ANSWER ===")
            print(final_msg)
            return final_msg

        else:
            # Unknown action
            message = "Unknown action. Use 'run_command' or 'finish'."
        #print(message)

    print("\nMax steps reached without 'finish'.")
    return None

if __name__ == "__main__":
    goal = input("Describe what you want the agent to do: ")
    agent_loop(goal)

Describe what you want the agent to do: Describe the files you have acces to

=== Step 1 ===
SENT MESSAGE: Describe the files you have acces to
RAW GPT REPLY: {"action": "run_command", "command": "ls -F"}

Executing command: 'ls -F'

=== Step 2 ===
SENT MESSAGE: Here is the result of your command:
Command: ls -F
Return code: 0
STDOUT:
add.py
ML-CyberSec-2025-Lab2-Public/
radare2/
sample_data/

STDERR:
(empty)
RAW GPT REPLY: {"action": "finish", "message": "I have access to the following files and directories:\n- `add.py`: A Python script.\n- `ML-CyberSec-2025-Lab2-Public/`: A directory, likely containing public lab materials for ML-CyberSec-2025 Lab 2.\n- `radare2/`: A directory, likely containing the radare2 reverse engineering framework or related files.\n- `sample_data/`: A directory, likely containing sample data."}

=== FINAL ANSWER ===
I have access to the following files and directories:
- `add.py`: A Python script.
- `ML-CyberSec-2025-Lab2-Public/`: A directory, likely containi